In [1]:
import csv
import urllib
import arcgis
import geopandas as gpd
from arcgis.gis import GIS
import numpy as np
from shapely.geometry import Point
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features.find_locations import find_existing_locations
from arcgis.features import SpatialDataFrame
from arcgis.features import use_proximity
from arcgis.geoenrichment import *
import pandas as pd
from arcgis.features.manage_data import merge_layers
from arcgis.geoenrichment import standard_geography_query, enrich
import seaborn as sns;
sns.set(color_codes=True)
import math
import matplotlib.pyplot as plt
from arcgis.features import summarize_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [54]:
# gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=c74bf4d93f514f0296c1ffd6dda71a73", client_id='S9hCtNsAKhoTpS87')
gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=01cee8d17e2446739d57e7714c693624", client_id='yYvN0h2gOFqtCZRA')
print("Successfully logged in as: " + gis.properties.user.username)
# print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=yYvN0h2gOFqtCZRA&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=ePPLjmztrbm7GGdqZWxLJzeggtwgP7
Enter code obtained on signing in using SAML: ········


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Successfully logged in as: yim095_UCSDOnline


In [11]:
# all the feature layer
polyly=gis.content.get('55ae7b83821947618ed692f6de4fe8c8')
pointsly=gis.content.get('1bef20c5bf4c46b897ce6a5788d20221')
centroidly=gis.content.get('c5799774da6c4271b00e8a50011f8da5')

# convert into SEDF
poly=pd.DataFrame.spatial.from_layer(polyly.layers[0])
points=pd.DataFrame.spatial.from_layer(pointsly.layers[0])
centroid=pd.DataFrame.spatial.from_layer(centroidly.layers[0])

# read the updated csv
update1=pd.read_csv('https://raw.githubusercontent.com/opensandiego/sandiego-covid-data/master/sd_daily_city_summary.csv')

In [13]:
# clean updated csv
index=update1.loc[update1['city'].isin (['other ','unknown* ', 'total san diego county residents '])].index
update=update1.drop(index,axis=0).reset_index(drop=True)
update['city']=update['city'].str.lower()
update['city']=update['city'].str.replace(' ', '')


In [14]:
# creat a new dataframe 
newup=pd.DataFrame(update['date'].unique()).T
newup.columns=newup.iloc[0,:]
newup.drop(0,inplace=True)
newup['city']=update['city'].unique()


In [15]:
# implement the new updated dataset with comfirmed cases
for i in range(len(update)):
    #get information from update dataframe
    city=update.loc[i,'city']
    date=update.loc[i,'date']
    value=update.loc[i,'confirmed_cases']
    
    #set value in new table
    index=newup.loc[newup['city']==city].index
    newup.loc[index,date]=value
newup.head()

,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-05,2020-04-07,city
0,11,12,14,13,16,15,18,20,20,24,27,30,32,42,42,carlsbad
1,4,4,6,7,13,17,21,24,24,32,38,52,65,104,120,chulavista
2,2,4,5,5,5,5,6,6,6,6,5,5,5,8,8,delmar
3,2,2,7,8,13,19,NaN,29,29,34,36,38,43,74,85,elcajon
4,4,5,5,7,8,12,14,18,18,19,20,24,25,30,31,encinitas


In [16]:
# implement the polygon layer with undated dataset
newpoly=poly[['FID','COom_adj','SUM_COUNT_','Shape__Length','Shape__Area','SHAPE']]
newpoly=newpoly.rename(columns={'COom_adj':'city','SUM_COUNT_':'zip_included'})
newpoly = newpoly.set_index(['FID'])
newpoly['city']=newpoly['city'].str.lower()
newpoly['city']=newpoly['city'].str.replace(' ', '')

newpoly = pd.merge(newpoly, newup, on='city',how='left')
newpoly.head()

,city,zip_included,Shape__Length,Shape__Area,SHAPE,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-05,2020-04-07
0,bonita,1,34253.928089,3.584046e+07,"{'rings': [[[-13022166.3169882, 3853307.240272...",NaN,NaN,1,1,1,1,2,2,2,4,3,6,6,8,11
1,carlsbad,4,56120.158052,1.444238e+08,"{'rings': [[[-13053082.0181245, 3916226.877610...",11,12,14,13,16,15,18,20,20,24,27,30,32,42,42
2,chulavista,5,84343.620753,2.664249e+08,"{'rings': [[[-13018505.2168385, 3853364.165457...",4,4,6,7,13,17,21,24,24,32,38,52,65,104,120
3,coronado,3,42455.007199,5.108144e+07,"{'rings': [[[-13038884.686413, 3851131.2618482...",NaN,NaN,NaN,0,0,NaN,1,1,1,1,1,2,2,4,4
4,delmar,1,29513.341671,2.436836e+07,"{'rings': [[[-13048633.6831031, 3892747.514411...",2,4,5,5,5,5,6,6,6,6,5,5,5,8,8


In [17]:
# add new column population_at_risk and infection per 10000
risk=pd.read_excel('Selected Conditions by Zip2017_04.07.20.xlsx')

In [ ]:
risk

In [ ]:
#newpoly.spatial.to_featurelayer(title='update poly',gis=gis,tags='dsc198')

In [55]:
uc=update['city'].unique()
pc=newpoly['city'].unique()
list(set(pc) - set(uc))

['campo',
 'sanclemente',
 'jacumba',
 'pala',
 'pinevalley',
 'ocotillo',
 'potrero',
 'palomarmountain',
 'santaysabel',
 'mountlaguna',
 'camppendleton',
 'guatay',
 'julian',
 'cardiffbythesea',
 'boulevard',
 'dulzura',
 'tecate',
 'aguanga',
 'bonsall',
 'warnersprings']

In [63]:
city = poly[['COom_adj','SHAPE']]


In [64]:
sd_zipcode = gis.content.get("1fa5dfbe76e5485faa90bdfed3a246c8")
sd_zipcode_sdf = pd.DataFrame.spatial.from_layer(sd_zipcode.layers[0])
zipcode = sd_zipcode_sdf[['ZIP','SHAPE','COMMUNITY']]

In [65]:
c = city['COom_adj'].unique()
z = zipcode['COMMUNITY'].unique()

In [66]:
city = city.rename(columns={'COom_adj':'COMMUNITY'})
city_with_zip = city.merge(zipcode, on='COMMUNITY')
city_with_zip = city_with_zip.drop('SHAPE_y',axis = 1)
city_with_zip = city_with_zip.rename(columns = {'SHAPE_x':'SHAPE'})
city_with_zip.head()

,COMMUNITY,SHAPE,ZIP
0,Bonita,"{'rings': [[[-13022166.3169882, 3853307.240272...",91902
1,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92008
2,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92009
3,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92010
4,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92011
